In [1]:
import sqlite3 as sq
import pandas as pd
import numpy as np
import os

## read data from file

In [2]:
df      = pd.read_csv('XETRA.csv')
df      = df.drop_duplicates(subset='ISIN',keep='first')
ctuples = [tuple(x) for x in df.values]

In [3]:
class Balance:
    # morning star balance is always in millions
    
    def __init__(self):
        self.currency = 'NON'
    
    def get_units(self):
        self.currency = self.data.columns[0].split('.')[1].split()[0]
        self.factors  = b.data.columns[0].split('in')[-1].split()[0]
        
    def readBalance(self,filename):
        self.data = pd.read_csv(filename,skiprows=1)
        self.get_units()

In [4]:
b = Balance()
b.readBalance('XETRA_ms_balance/R6C3-ROYAL-DUTCH-SHELL-B-EO-07.csv')
b.data.columns = ['Entity',2011,2012,2013,2014,2015]


In [5]:
b_categories = b.data.Entity.values.tolist()

In [6]:
b.currency

'USD'

In [68]:
for i,v in enumerate(b.data.Entity.values):
    
    categories = ['Current assets','Non-current assets',"Liabilities and stockholders' equity", 
                  "Current liabilities"]
    
    if v in categories:
        print v, i

Current assets 1
Non-current assets 10
Liabilities and stockholders' equity 23
Current liabilities 25


## read the different possible currencies

In [6]:
currencies = []
factors    = []

for filename in os.listdir('XETRA_ms_balance/'):
    if filename.endswith(".csv"): 
        try:
            b = Balance()
            b.readBalance('XETRA_ms_balance/'+filename)
            currencies.append(b.currency)
            factors.append(b.factors)
        except:
            pass
#         print filename
    else:
        continue
        
currencies = np.array(currencies)
factors    = np.array(factors)

In [7]:
# list of all currencies
currencies = np.unique(currencies)

## Possible entries in Equity

In [8]:
def show_entries_in_category(startline,endline,nmax=0,silent=False):
    pline    = False
    entities = []
    n        = 1
    
    #print startline
    #print endline
    
    for filename in os.listdir('XETRA_ms_balance/'):
        if filename.endswith(".csv"):
#             print filename
            n+=1
            if n==nmax:
                break
            try:
#             if True:
                with open('XETRA_ms_balance/'+filename) as f:
                    pline = False
                    for line in f:
#                         print line
                        if line==endline+"\n":
                            pline = False
                            
                        if pline and line.split(',')[0].replace('\n','').replace('"','') not in entities:
                            if not silent:
                                print line.split(',')[0]        
                            entities.append(line.split(',')[0].replace('\n','').replace('"',''))
                            
                        if line==startline+"\n":
                            pline=True
            except:
                pass
        else:
            continue    
    
    return np.array(entities)

In [9]:
balance_entries = {}

In [10]:
balance_entries['Current Assets'] = show_entries_in_category("Current assets",
                                                             "Non-current assets",
                                                             nmax=0,
                                                             silent=True)

In [11]:
balance_entries['Non-Current Assets'] = show_entries_in_category("Non-current assets",
                                                                 "Liabilities and stockholders' equity",
                                                                 nmax=0,
                                                                 silent=True)

In [12]:
balance_entries['Stockholders Equity'] = show_entries_in_category("Stockholders' equity","---",
                                                                  nmax=0,
                                                                  silent=True)

In [13]:
balance_entries['Current liabilities'] = show_entries_in_category("Current liabilities",
                                                                  "Non-current liabilities",
                                                                  silent=True)

In [14]:
balance_entries['Non-current liabilities'] = show_entries_in_category("Non-current liabilities","",silent=True)

## Define Categories and sub-categories

In [15]:


balance_entries['Non-current liabilities']

array(['Long-term debt', 'Deferred taxes liabilities', 'Minority interest',
       'Other long-term liabilities', 'Total non-current liabilities',
       'Total liabilities', "Stockholders' equity", 'Preferred stock',
       'Common stock', 'Additional paid-in capital', 'Retained earnings',
       'Treasury stock', 'Accumulated other comprehensive income',
       "Total stockholders' equity",
       "Total liabilities and stockholders' equity", 'Capital leases',
       "Total Stockholders' equity", 'Deferred tax liabilities',
       'Pensions and other benefits', 'Accrued liabilities',
       'Deferred revenues', 'Pensions and other postretirement benefits',
       'Regulatory liabilities', "stockholders' equity", 'General Partner',
       'Other Equity'], 
      dtype='|S42')

In [16]:
class BalanceSQL:
    
    def __init__(self):
        self.filename = 'Undefined'
        self.currency = 'NON'
        self.factor   = 'ONE'
        pass

    def check_filename(self):
        if self.filename=='Undefined':
            print 'filename not defined. please indicate filename'
            
    def get_isin(self):
        self.ticker = self.filename.split('/')[-1]
        self.ticker = self.ticker.split('-')[0]
        
        self.name,self.isin,self.ticker = np.array(ctuples)[np.array(ctuples)[:,2]==self.ticker][0]
        pass
    
    
    
    def get_header(self):
        self.check_filename()
        
        with open(self.filename) as f:
            for line in f:        
                if 'Fiscal' in line:
                    
                    self.years = line.replace('\n','').split(',')[1:]                
                    self.years = [int(y.split('-')[0]) for y in self.years]
                    
                    self.currency = line.split('.')[1].split()[0]
                    self.factor   = line.split('in')[-1].split()[0]

    def return_SQL_tupel_from_line(self,line,category,subcategory,year):
                
        line      = line.replace('property,','property').replace('"','')
        line      = line.replace('Property,','Property').replace('"','')
                
        line      = line.replace('\n','').split(',')
        
        entryname = line[0] 
        values    = line[1:]
        
        year_indx = np.where(np.array(self.years)==year)[0][0]
        
        try:
            values    = np.array(values).astype('float')
        except:
            for i,v in enumerate(values):
                if v=='':
                    values[i] = 0
                else:
                    values[i] = int(v)
                    
        values = np.array(values)
                
        return self.name, self.ticker, self.isin, self. currency, year, category, subcategory, entryname, values[year_indx]                    
    
    def get_category(self,line,ini_category,ini_subcategory):
        
        category    = "None"
        subcategory = "None"
        
        if line=="Assets\n":
            category    = "Assets"
            subcategory = "Assets"
            
        if line=="Current assets\n":
            category    = "Assets"
            subcategory = "Current assets"            

        if line=="Non-current assets\n":
            category    = "Assets"
            subcategory = "Non-current assets"    

        if "Total assets" in line:
            category    = "Assets"
            subcategory = "Total assets"
            
        if line=="Liabilities\n":
            category    = "Liabilities and equity"
            subcategory = "Liabilities"

        if line=="Current liabilities\n":
            category    = "Liabilities and equity"
            subcategory = "Current liabilities"

        if line=="Non-current liabilities\n":
            category    = "Liabilities and equity"
            subcategory = "Non-current liabilities"

        if "Total liabilities" in line and 'equity' not in line:
            category    = "Liabilities and equity"
            subcategory = "Total liabilities"                

        if line=="Stockholders' equity\n":
            category    = "Liabilities and equity"
            subcategory = "Equity"

        if "Total liabilities and stockholders' equity" in line:
            category    = "Total liabilities and equity"
            subcategory = "Total liabilities and equity"  
        
        if category!="None" and subcategory!="None":
            return category, subcategory
        else:
            return ini_category, ini_subcategory
    
    
    def return_balance_with_categories(self,year,silent=False,show_sql=False):

        balanceitems = []
        
        category    = "None"
        subcategory = "None"        
        
        self.get_isin()
        
        if show_sql:
            print '-->  SQL BALANCE INFORMATION'
            print '-->  Currency  ', self.currency
            print '-->  Factor    ', self.factor
            print '-->  Year      ', year
            print '-->  Name      ', self.name
            print '-->  ISIN      ', self.isin
            print '-->  Ticker    ', self.ticker
            print 
        with open(self.filename) as f:
            
            for line in f:

                category, subcategory = self.get_category(line,category,subcategory)
                    
                if category!="None" and line !='':

                    line = line.replace('property,','property')
                    line = line.replace('Property,','Property')
                    line = line.replace('"','')
                    
                    
                    if len(line.split(','))>1:
                        tupel = self.return_SQL_tupel_from_line(line,category,subcategory,year)
                        balanceitems.append(tupel)
                        if show_sql:
                            print tupel 
                            
                    line = line.replace('property,','property').replace('"','')
                    line = line.replace("'","")

                    try:
                        if not silent:
                            print '%29s  %29s  %42s  %10i'  % (category, subcategory,line.split(',')[0],
                                                                    int(line.split(',')[1]))
                    except:
                        pass
                    
                    
                if 'Total assets' in line and not silent:
                    print 

        return balanceitems

## Examples for single stocks

In [17]:
b = BalanceSQL()
b.filename = 'XETRA_ms_balance/R6C3-ROYAL-DUTCH-SHELL-B-EO-07.csv'
b.get_header()
tupels = b.return_balance_with_categories(2013,silent=True,show_sql=True)

-->  SQL BALANCE INFORMATION
-->  Currency   USD
-->  Factor     millions
-->  Year       2013
-->  Name       ROYAL DUTCH SHELL B EO-07
-->  ISIN       GB00B03MM408
-->  Ticker     R6C3

('ROYAL DUTCH SHELL B EO-07', 'R6C3', 'GB00B03MM408', 'USD', 2013, 'Assets', 'Current assets', 'Cash and cash equivalents', 9696.0)
('ROYAL DUTCH SHELL B EO-07', 'R6C3', 'GB00B03MM408', 'USD', 2013, 'Assets', 'Current assets', 'Total cash', 9696.0)
('ROYAL DUTCH SHELL B EO-07', 'R6C3', 'GB00B03MM408', 'USD', 2013, 'Assets', 'Current assets', 'Receivables', 50357.0)
('ROYAL DUTCH SHELL B EO-07', 'R6C3', 'GB00B03MM408', 'USD', 2013, 'Assets', 'Current assets', 'Inventories', 30009.0)
('ROYAL DUTCH SHELL B EO-07', 'R6C3', 'GB00B03MM408', 'USD', 2013, 'Assets', 'Current assets', 'Prepaid expenses', 6836)
('ROYAL DUTCH SHELL B EO-07', 'R6C3', 'GB00B03MM408', 'USD', 2013, 'Assets', 'Current assets', 'Other current assets', 6445)
('ROYAL DUTCH SHELL B EO-07', 'R6C3', 'GB00B03MM408', 'USD', 2013, 'Assets', 'C

In [18]:
b = BalanceSQL()
b.filename = 'XETRA_ms_balance/MUV2-MUENCH.RUECKVERS.VNA-O.N..csv'
b.get_header()
tupels = b.return_balance_with_categories(2013,silent=True,show_sql=True)

-->  SQL BALANCE INFORMATION
-->  Currency   EUR
-->  Factor     millions
-->  Year       2013
-->  Name       MUENCH.RUECKVERS.VNA O.N.
-->  ISIN       DE0008430026
-->  Ticker     MUV2

('MUENCH.RUECKVERS.VNA O.N.', 'MUV2', 'DE0008430026', 'EUR', 2013, 'Assets', 'Assets', 'Cash and cash equivalents', 2820.0)
('MUENCH.RUECKVERS.VNA O.N.', 'MUV2', 'DE0008430026', 'EUR', 2013, 'Assets', 'Assets', 'Deferred policy acquisition costs', 9542)
('MUENCH.RUECKVERS.VNA O.N.', 'MUV2', 'DE0008430026', 'EUR', 2013, 'Assets', 'Assets', 'Property and equipment', 3762.0)
('MUENCH.RUECKVERS.VNA O.N.', 'MUV2', 'DE0008430026', 'EUR', 2013, 'Assets', 'Assets', 'Goodwill', 4672.0)
('MUENCH.RUECKVERS.VNA O.N.', 'MUV2', 'DE0008430026', 'EUR', 2013, 'Assets', 'Assets', 'Other assets', 233492.0)
('MUENCH.RUECKVERS.VNA O.N.', 'MUV2', 'DE0008430026', 'EUR', 2013, 'Assets', 'Total assets', 'Total assets', 254288.0)
('MUENCH.RUECKVERS.VNA O.N.', 'MUV2', 'DE0008430026', 'EUR', 2013, 'Liabilities and equity', 'Liab

## Populate SQL Database with Balance Data

In [247]:
failed = []

conn = sq.connect('database/stocks.db')
c    = conn.cursor()

c.execute('''CREATE TABLE balance 
    (
    name        TEXT NOT NULL,
    ticker      TEXT NOT NULL,
    isin        TEXT,    
    currency    TEXT,
    year        INT  NOT NULL,
    category    TEXT NOT NULL,
    subcategory TEXT NOT NULL,
    entity      TEXT NOT NULL,
    amount      REAL NOT NULL
    );''')

for filename in os.listdir('XETRA_ms_balance/'):
    if filename.endswith(".csv"): 
        b = BalanceSQL()
        b.filename = 'XETRA_ms_balance/'+filename
        b.get_header()
        try:
            for y in b.years:
                tupels = b.return_balance_with_categories(y,silent=True,show_sql=False)
                c.executemany('INSERT INTO balance VALUES (?,?,?,?,?,?,?,?,?)', tupels)
        except:
            failed.append(filename)

conn.commit()
conn.close()



## LOAD KEYRATIOS

In [2]:
df = pd.read_csv('XETRA_ms_keyratios/MUV2-MUENCH.RUECKVERS.VNA-O.N..csv',skiprows=2)

In [3]:
df.columns

Index([u'Unnamed: 0', u'2006-12', u'2007-12', u'2008-12', u'2009-12',
       u'2010-12', u'2011-12', u'2012-12', u'2013-12', u'2014-12', u'2015-12',
       u'TTM'],
      dtype='object')

In [4]:
newcols = []
for i in df.columns:
    if 'Unnamed' in i:
        newcols.append('Parameter')        
    elif i=='TTM':
        newcols.append(2016)
    else:
        newcols.append(int(i.split('-')[0]))


In [5]:
newcols

['Parameter', 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016]

In [6]:
df.columns = newcols

In [7]:
df[0:4]

,Parameter,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016
0,Revenue EUR Mil,"49,880","53,124","61,383","59,583","59,111","60,688","72,286","56,718","55,634","55,985","54,911"
1,Gross Margin %,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Operating Income EUR Mil,"5,477","5,057","3,834","4,721","3,978","1,180","5,349","4,398","4,027","4,819","4,629"
3,Operating Margin %,11.0,9.5,6.2,7.9,6.7,1.9,7.4,7.8,7.2,8.6,8.4


In [12]:
df[df.Parameter=='Revenue EUR Mil']

,Parameter,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016
0,Revenue EUR Mil,"49,880","53,124","61,383","59,583","59,111","60,688","72,286","56,718","55,634","55,985","54,911"
